In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline

In [4]:
import os
import numpy as np
import pandas as pd
import glob
import math

In [5]:
class HitPredictor(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=20, output_dim=20,
                n_lstm_layers=1):
        super(HitPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = 1
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_lstm_layers,
                           batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.lstm.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, x):
        input_size = x.size()
        x, self.hidden = self.lstm(x, self.hidden)        
        x = (self.fc(x.contiguous().view(-1, x.size(-1)))
             .view(input_size[0], input_size[1], -1))
        return x

In [6]:
from utils import make_uID
det = pd.read_csv('input/detectors.csv')
det.head()
hits_pd = make_uID(det)
hits_pd.shape

(18728, 4)

### load one event

In [7]:
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join('input/train_1', event_prefix))

In [8]:
hits_with_uID = pd.merge(hits, hits_pd, on=['volume_id', 'layer_id', 'module_id'])
from utils import get_features
hits_final = get_features(hits_with_uID)
hits_final.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,uID,x2,y2,z2,r2,r,eta,phi,absZ
0,1,-64.409897,-7.163700,-1502.5,7,2,1,0,-0.042829,-0.004763,-23.184208,64.807045,1503.896973,-3.837108,-3.030827,1502.5
1,2,-55.336102,0.635342,-1502.5,7,2,1,0,-0.036804,0.000423,-27.150467,55.339748,1503.518799,-3.994889,3.130112,1502.5
2,3,-83.830498,-1.143010,-1502.5,7,2,1,0,-0.055707,-0.000760,-17.921406,83.838287,1504.837158,-3.579932,-3.127959,1502.5
3,4,-96.109100,-8.241030,-1502.5,7,2,1,0,-0.063835,-0.005474,-15.576118,96.461777,1505.593262,-3.439919,-3.056055,1502.5
4,5,-62.673599,-9.371200,-1502.5,7,2,1,0,-0.041676,-0.006232,-23.709833,63.370335,1503.835815,-3.859459,-2.993168,1502.5


In [9]:
filtered_truth = truth[ (truth['weight'] > 5e-7 ) & (truth['particle_id'] != 0) ]
training = hits_final.merge(filtered_truth, on='hit_id')[['uID', 'particle_id']]

In [18]:
input_ = torch.from_numpy(training[training['particle_id'] == 297237712845406208]['uID'].values)
print(input_)
print(input_.size())

tensor([   0,  108,  216,  324,  429,  432,  537,  540,  645,  648,
         753])
torch.Size([11])


In [13]:
training[training['particle_id'] == 297237712845406208]['uID'].values

array([  0, 108, 216, 324, 429, 432, 537, 540, 645, 648, 753])

In [19]:
torch.nn.utils.rnn.pack_padded_sequence(input_, [20])

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

In [34]:
rnn = HitPredictor()

In [ ]:
input_ = torch.tensor([training[training['particle_id'] == 297237712845406208]])

In [12]:
rnn = nn.LSTM(10, 20, 1)
input = torch.randn(3, 3, 10)
h0 = torch.randn(1, 3, 20)
c0 = torch.randn(1, 3, 20)
output, hn = rnn(input, (h0, c0))
print(output)

tensor([[[-0.6052,  0.0523, -0.0689,  0.3538, -0.2065,  0.2301,  0.1169,
           0.2881, -0.1148, -0.0159,  0.0207,  0.0215,  0.0508, -0.1013,
           0.0696, -0.2852,  0.1259, -0.0748,  0.0710,  0.0454],
         [-0.0658,  0.0487,  0.1512, -0.1353, -0.1221,  0.1889, -0.0168,
           0.1365,  0.0165, -0.1805, -0.3716,  0.2688, -0.0126,  0.1646,
           0.4330, -0.1390, -0.0259,  0.0650, -0.2073, -0.4505],
         [ 0.1131, -0.2102, -0.0839,  0.1776, -0.0707, -0.3263,  0.0885,
          -0.0328,  0.2581, -0.0846, -0.0050, -0.1796, -0.2992,  0.1977,
           0.0100,  0.0514,  0.4538,  0.1063, -0.4818,  0.3612]],

        [[-0.2573, -0.0517,  0.0031,  0.1839, -0.0588,  0.1845,  0.0734,
           0.2018, -0.0919,  0.0712, -0.1035, -0.0663,  0.1174, -0.1251,
           0.1845, -0.1601, -0.1314, -0.0414,  0.1870,  0.1128],
         [-0.0087, -0.0795,  0.1371, -0.0937,  0.0446,  0.1415,  0.0561,
           0.1343,  0.0221, -0.0676, -0.2361,  0.0357,  0.0726,  0.1188,
        

In [8]:
output.size()

torch.Size([5, 3, 20])